In [1]:
!pip install arxiv

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6105 sha256=4ae3741d3fe3235e9c7469daac4803429a163eda84f95f87dc9efb172e6c8940
  Stored in directory: c:\users\agoel\appdata\local\pip\cache\wheels\f0\69\93\a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k

   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   ------------- -------------------------- 1/3 [feedparser]
   -------------------------- ------------- 2/3 [arxiv]
   ---------------------------------------- 3/3 [arxiv]



  DEPRECATION: Building 'sgmllib3k' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'sgmllib3k'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [19]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") 

In [12]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
embedding_instance = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents,embedding_instance)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000228BE042500>, search_kwargs={})

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith_search",
                        "Search for imformation about LangSmith.For any questions about LangSmith, you should search wikipedia")


In [15]:
retriever_tool.name

'langmsith_search'

In [18]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

api_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper)
arxiv.name

'arxiv'

In [20]:
tools = [wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\AI_Python\\Agents\\LangChain\\la-env\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=300)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=300)),
 Tool(name='langmsith_search', description='Search for imformation about LangSmith.For any questions about LangSmith, you should search wikipedia', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000228F76BF0A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore

In [21]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
from langchain.agents import Tool,AgentExecutor, initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21")
agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    #prompt,
    verbose=True
)
agent
#agent.invoke("Tell me about Langsmith")
#print(agent.keys)
# agent.invoke({
#     "input": "Tell me about Langsmith",
#     "page_content": "LangSmith is a platform for debugging and evaluating LLM apps."  # Replace with actual retrieved content
# })


AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['', 'agent_scratchpad', 'input', 'page_content'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangmsith_search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'GoogleGenerativeAIEmbeddings\'], vectorstore=<lan

In [43]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor
#agent_executor.invoke({"input":"Tell me about Langsmith"})

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=AgentExecutor(verbose=True, tags=['zero-shot-react-description'], agent=ZeroShotAgent(llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['', 'agent_scratchpad', 'input', 'page_content'], input_types={}, partial_variables={}, template='Answer the following questions as best you can. You have access to the following tools:\n\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nlangmsith_search(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAI

In [44]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...


> Entering new None chain...


TypeError: Agent.plan() got multiple values for argument 'intermediate_steps'



> Entering new AgentExecutor chain...


> Entering new None chain...


TypeError: Agent.plan() got multiple values for argument 'intermediate_steps'